In [1]:
import pandas as pd

from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'

sheet_name = 'zavala'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=zavala'

In [5]:
output_file_path = '../data/labels/gt-bands.xlsx'

In [6]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)

In [7]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
#drop any columns that have 'Unnamed:' in the column name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,SandSnapID,SandSnap Photo,D16,D50,D90,Sieve Results,S-D16,S-D50,S-D90,SSE
0,Zavala-0,-24.752861,34.799964,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1488,PXL_20230509_092847602.jpg,0.489,0.681,1.108,NaN,NaN,NaN,NaN,1.389441
1,Zavala-1,-24.752831,34.800144,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1503,NaN,0.512,0.713,1.161,NaN,NaN,NaN,NaN,1.455484
2,Zavala-2,-24.752506,34.800969,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1510,NaN,0.481,0.671,1.091,NaN,NaN,NaN,NaN,NaN
3,Zavala-3,-24.752506,34.800969,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1511,NaN,0.471,0.657,1.070,NaN,NaN,NaN,NaN,1.341041
4,Zavala-4,-24.752486,34.801033,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1512,NaN,0.501,0.698,1.136,NaN,NaN,NaN,NaN,1.424325
5,Zavala-5,-24.752736,34.801097,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1513,NaN,0.554,0.772,1.256,NaN,NaN,NaN,NaN,1.574940
6,Zavala-6,-24.753103,34.799458,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1514,NaN,0.556,0.775,1.260,NaN,NaN,NaN,NaN,1.580304
7,Zavala-7,-24.753306,34.798517,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1516,NaN,0.506,0.704,1.145,NaN,NaN,NaN,NaN,1.436202
8,Zavala-8,-24.753467,34.798022,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1517,NaN,0.545,0.759,1.234,NaN,NaN,NaN,NaN,1.547857
9,Zavala-9,-24.753653,34.797492,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1518,NaN,0.548,0.762,1.240,NaN,NaN,NaN,NaN,1.555168


In [8]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [9]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [10]:
output = df.copy()

In [11]:
INDEX = 0

In [59]:
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [60]:
#Uncomment to change the date of the image
# output['Date'].iloc[INDEX] = '2022-10-14'
# output['ID'].iloc[INDEX] = 'ns-49'

In [61]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='median', sampling_buffer_m=sampling_buffer_m, 
                                 max_search_window_months=3, median_samples=5,roi_buffer_m=5000, obia=True)

Index:  14  ID:  Zavala-14 Class:  Sand  Site:  coastal
Search window from 2023-02-11 to 2023-08-11


## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function.
It will also overwrite just the sheet if the same exists, but will preserve the other sheets in the book

In [62]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
# fm.write_to_excel(output, output_file_path, sheet_name)
with pd.ExcelWriter(output_file_path,engine="openpyxl",mode="a",if_sheet_exists="replace") as writer:
    output.to_excel(writer,sheet_name=sheet_name,index=False)    
display(output.iloc[INDEX])
INDEX += 1

New marker accepted
Kept Observation


ID                    Zavala-14
Latitude             -24.749152
Longitude             34.800027
Class                      Sand
Date                 2023-05-11
Site                    coastal
Source                   Author
Country              Mozambique
Notes                       NaN
SandSnapID                 1524
SandSnap Photo              NaN
D16                       0.517
D50                       0.722
D90                       1.175
Sieve Results               NaN
S-D16                       NaN
S-D50                       NaN
S-D90                       NaN
SSE                         NaN
B2_mean              609.531555
B3_mean              961.099121
B4_mean              1159.48645
B8_mean              2846.28833
B8A_mean            3070.171143
B11_mean            3044.189209
B12_mean            2362.513428
VV_mean              -11.930748
VH_mean              -17.889772
mTGSI_mean             0.006867
BSI_mean               0.096578
NDWI_mean             -0.495506
keep    

## Final Output

In [32]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,SandSnapID,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,Zavala-0,-24.752861,34.799964,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1488,...,3992.821045,3775.798096,-15.005158,-23.35412,0.118486,0.156983,-0.214037,True,False,1
1,Zavala-1,-24.752831,34.800144,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
2,Zavala-2,-24.752435,34.800861,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1510,...,3992.821045,3775.798096,-15.005158,-23.35412,0.118486,0.156983,-0.214037,True,True,1
3,Zavala-3,-24.752506,34.800969,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
4,Zavala-4,-24.752486,34.801033,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
5,Zavala-5,-24.752736,34.801097,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
6,Zavala-6,-24.753103,34.799458,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1514,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
7,Zavala-7,-24.753306,34.798517,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1516,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
8,Zavala-8,-24.753467,34.798022,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
9,Zavala-9,-24.753653,34.797492,Sand,2023-05-11,coastal,Author,Mozambique,NaN,1518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
